In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [13]:
# MNIST example from TensorFlow tutorials

mnist = tf.keras.datasets.mnist

# Load and scale data
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y) = mnist.load_data()
mnist_train_x, mnist_test_x = mnist_train_x / 255.0, mnist_test_x / 255.0

# Creat simple NN
mnist_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
mnist_model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy'])

# Fit model
mnist_model.fit(mnist_train_x, mnist_train_y, epochs=5)

# Test model 
mnist_model.evaluate(mnist_test_x, mnist_test_y)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2963 - accuracy: 0.9125
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1430 - accuracy: 0.9580
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1085 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0877 - accuracy: 0.9731
Epoch 5/5
313/313 [==============================] - 0s 743us/step - loss: 0.0731 - accuracy: 0.9772


[0.07309569418430328, 0.9771999716758728]

In [12]:
# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

# Load dataset, remove rows with missing info
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True).dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

# Convert Origin to one-hot
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')

# Split train/test and split labels
mpg_train_x = dataset.sample(frac=0.8, random_state=0)
mpg_test_x = dataset.drop(mpg_train_x.index)
mpg_train_y = mpg_train_x.pop('MPG')
mpg_test_y = mpg_test_x.pop('MPG')

# Scale data
mpg_norm = tf.keras.layers.Normalization(axis=-1)
mpg_norm.adapt(np.array(mpg_train_x))

# Build model
def build_and_compile_model(norm):
  model = tf.keras.Sequential([
      norm,
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

mpg_model = build_and_compile_model(mpg_norm)

mpg_model_history = mpg_model.fit(mpg_train_x, mpg_train_y, validation_split=0.1, epochs=100, verbose=0)
mpg_model.evaluate(mpg_test_x, mpg_test_y, verbose=0)

1.5733596086502075